In [1]:
import pandas as pd
import numpy as np

# 1. **데이터 설명**
1. pathway_train, pathway_test, pathway_valid : CCLE에서 pathway score 계산 한 데이터
2. ic_train, ic_test, ic_val : 민감군, 저항군 라벨
3. ccle : 유전자 raw data -> 민감군, 저항군 간 유전자 발현량이 가장 큰 유전자 3개를 genes에 저장
4. tcga: 환자 데이터로 약물 민간/저항 라벨 부착 예정

In [2]:
pathway_train = pd.read_csv("train_pathway_score_REFAMETINIB.csv")
pathway_test = pd.read_csv("test_pathway_score_REFAMETINIB.csv")
pathway_valid = pd.read_csv("valid_pathway_score_REFAMETINIB.csv")

ic_train = pd.read_csv("ic_train_REFAMETINIB.csv")
ic_test = pd.read_csv("ic_test_REFAMETINIB.csv")
ic_val = pd.read_csv("ic_valid_REFAMETINIB.csv")

ccle = pd.read_csv("C:/Users/User/BAF-의료/data/#_filtered_CCLE_gene_expression.csv")
ccle.index = ccle["Unnamed: 0"]

tcga_path = pd.read_csv("tcga_pathway_score_REFAMETINIB.csv")
tcga = pd.read_csv("C:/Users/User/BAF-의료/data/TCGA_final_0419.csv")


In [3]:
pathway_train.set_index("SampleID", inplace= True)
pathway_test.set_index("SampleID", inplace= True)
pathway_valid.set_index("SampleID", inplace= True)

ic_train.set_index("Unnamed: 0", inplace=True)
ic_test.set_index("Unnamed: 0", inplace=True)
ic_val.set_index("Unnamed: 0", inplace=True)

tcga_path.set_index("SampleID",inplace=True)
tcga.set_index("Unnamed: 0", inplace=True)

In [4]:
# DEG로 계산한 민감군, 저항군 간 차이가 큰 유전자 3가지
genes = ["LYZ", "MMP1","IL1B" ]

# 2. **전처리**
1. pathway score scale
2. pathway score에 대해 PCA 진행 후 90% 설명력까지 사용
3. train 기준으로 fjt하고 trest, valid, TCGA에는 transform만 해주기
4. ccle, tcga 유전자들도 sacle해주기
5. ic라벨 인코딩

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 스케일링 (z-score 정규화가 되어있더라도 PCA 전에 한 번 더 정규화 )
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pathway_train)
X_test_scaled = scaler.transform(pathway_test)
X_val_scaled = scaler.transform(pathway_valid)
tcga_path_scaled = scaler.transform(tcga_path)

# PCA 수행 (주성분 2개로 축소)
pca = PCA(n_components=0.90)
pathway_pca_train = pca.fit_transform(X_scaled)
pathway_pca_test  = pca.transform(X_test_scaled)
pathway_pca_valid = pca.transform(X_val_scaled)
pathway_pca_tcga = pca.transform(tcga_path_scaled)

In [6]:
pca_columns = [f"PC{i+1}" for i in range(pathway_pca_train.shape[1])]
pca_train_df_all = pd.DataFrame(pathway_pca_train, columns=pca_columns, index=pathway_train.index)
pca_train_df_all

pca_valid_df_all = pd.DataFrame(pathway_pca_valid, columns=pca_columns, index=pathway_valid.index)
pca_valid_df_all

pca_test_df_all = pd.DataFrame(pathway_pca_test, columns=pca_columns, index=pathway_test.index)
pca_test_df_all

pca_tcga_df_all = pd.DataFrame(pathway_pca_tcga, columns=pca_columns, index = tcga_path.index)

In [7]:
ccle = ccle[genes]
ccle_expr_train = ccle.loc[pathway_train.index]
ccle_log_train = np.log2(ccle_expr_train+1)

scaler = StandardScaler()
ccle_scaled_train = pd.DataFrame(scaler.fit_transform(ccle_log_train),
                          columns = ccle_expr_train.columns,
                          index = ccle_expr_train.index)

ccle_expr_valid = ccle.loc[pathway_valid.index]
ccle_log_valid = np.log2(ccle_expr_valid+1)

ccle_scaled_valid = pd.DataFrame(scaler.transform(ccle_log_valid),
                          columns = ccle_expr_valid.columns,
                          index = ccle_expr_valid.index)

ccle_expr_test = ccle.loc[pathway_test.index]
ccle_log_test = np.log2(ccle_expr_test+1)
ccle_scaled_test = pd.DataFrame(scaler.transform(ccle_log_test),
                          columns = ccle_expr_test.columns,
                          index = ccle_expr_test.index)

In [8]:
tcga = tcga[genes]
tcga_expr = tcga.loc[tcga.index]
tcga_log = np.log2(tcga_expr+1)

tcga_scaled = pd.DataFrame(scaler.transform(tcga_log),
                          columns=tcga_expr.columns,
                          index = tcga_expr.index)


In [9]:
X_train = pd.concat([pca_train_df_all, ccle_scaled_train], axis = 1)
X_valid = pd.concat([pca_valid_df_all, ccle_scaled_valid], axis = 1)
X_test = pd.concat([pca_test_df_all, ccle_scaled_test], axis = 1)
X_tcga = pd.concat([pca_tcga_df_all,tcga_scaled], axis = 1)

final_scaler = StandardScaler()
X_train_final = final_scaler.fit_transform(X_train)
X_valid_final = final_scaler.transform(X_valid)
X_test_final = final_scaler.transform(X_test)
X_tcga_final = final_scaler.transform(X_tcga)

In [10]:
# 숫자 라벨로 변환 (sensitive = 1, resistant = 0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(ic_train.iloc[:,0])
y_valid = le.fit_transform(ic_val.iloc[:,0])
y_test = le.fit_transform(ic_test.iloc[:,0])

# **3. 모델링**
1. 로지스틱, 랜덤포레스트, SVM, KNN으로 진행
2. 전처리를 R로 해야했기에 pipeline으로 묶을 수 없어 cross validation이 힘듬
3. 평가지표는 AUC, F1 score 중점으로 사용
4. 수동으로 튜닝 후 성능이 제일 좋은 모델 선택 후 TCGA 라벨 예측

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

In [12]:
## Logistic Regression
lr = LogisticRegression(C=0.01, max_iter=10000,class_weight="balanced")
lr.fit(X_train_final, y_train)
y_pred_lr = lr.predict(X_test_final)
y_proba_lr = lr.predict_proba(X_test_final)[:, 1]

## Random Forest
rf = RandomForestClassifier(n_estimators=250, max_depth=4, random_state=42,class_weight="balanced")
rf.fit(X_train_final, y_train)
y_pred_rf = rf.predict(X_test_final)
y_proba_rf = rf.predict_proba(X_test_final)[:, 1]

## Support Vector Machine (SVM)
svm = SVC(probability=True, random_state=42,class_weight="balanced")
svm.fit(X_train_final, y_train)
y_pred_svm = svm.predict(X_test_final)
y_proba_svm = svm.predict_proba(X_test_final)[:, 1]

## K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_final, y_train)
y_pred_knn = knn.predict(X_test_final)
y_proba_knn = knn.predict_proba(X_test_final)[:, 1]

# ── AdaBoost 추가 ──
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(
    n_estimators=50,
    random_state=42
)
ada.fit(X_train_final, y_train)

y_pred_ada   = ada.predict(X_test_final)
y_proba_ada  = ada.predict_proba(X_test_final)[:, 1]

In [13]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

print("===== Validation Set Evaluation =====")

# Logistic Regression - Validation
y_pred_lr_valid = lr.predict(X_valid_final)
y_proba_lr_valid = lr.predict_proba(X_valid_final)[:, 1]
print("\n--- Logistic Regression ---")
print(classification_report(y_valid, y_pred_lr_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_lr_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_lr_valid))

# Random Forest - Validation
y_pred_rf_valid = rf.predict(X_valid_final)
y_proba_rf_valid = rf.predict_proba(X_valid_final)[:, 1]
print("\n--- Random Forest ---")
print(classification_report(y_valid, y_pred_rf_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_rf_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_rf_valid))

# Support Vector Machine - Validation
y_pred_svm_valid = svm.predict(X_valid_final)
y_proba_svm_valid = svm.predict_proba(X_valid_final)[:, 1]
print("\n--- Support Vector Machine ---")
print(classification_report(y_valid, y_pred_svm_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_svm_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_svm_valid))

# K-Nearest Neighbors - Validation
y_pred_knn_valid = knn.predict(X_valid_final)
y_proba_knn_valid = knn.predict_proba(X_valid_final)[:, 1]
print("\n--- K-Nearest Neighbors ---")
print(classification_report(y_valid, y_pred_knn_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_knn_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_knn_valid))

# AdaBoost - Validation
y_pred_ada_valid   = ada.predict(X_valid_final)
y_proba_ada_valid  = ada.predict_proba(X_valid_final)[:, 1]
print("\n--- AdaBoost ---")
print(classification_report(y_valid, y_pred_ada_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_ada_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_ada_valid))

===== Validation Set Evaluation =====

--- Logistic Regression ---
              precision    recall  f1-score   support

   resistant       0.66      0.60      0.63        35
   sensitive       0.60      0.66      0.63        32

    accuracy                           0.63        67
   macro avg       0.63      0.63      0.63        67
weighted avg       0.63      0.63      0.63        67

ROC-AUC: 0.6982142857142858
Confusion matrix:
 [[21 14]
 [11 21]]

--- Random Forest ---
              precision    recall  f1-score   support

   resistant       0.71      0.57      0.63        35
   sensitive       0.62      0.75      0.68        32

    accuracy                           0.66        67
   macro avg       0.66      0.66      0.66        67
weighted avg       0.67      0.66      0.65        67

ROC-AUC: 0.6267857142857143
Confusion matrix:
 [[20 15]
 [ 8 24]]

--- Support Vector Machine ---
              precision    recall  f1-score   support

   resistant       0.62      0.57    

In [14]:
from sklearn.ensemble import VotingClassifier
# ── Soft Voting Ensemble ──
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('svm', svm),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft'
)
voting_clf.fit(X_train_final, y_train)

y_pred_voting  = voting_clf.predict(X_test_final)
y_proba_voting = voting_clf.predict_proba(X_test_final)[:, 1]

print("\n===== Soft Voting Ensemble =====")
print(classification_report(y_test, y_pred_voting, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_voting))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_voting))





===== Soft Voting Ensemble =====
              precision    recall  f1-score   support

   resistant       0.66      0.69      0.67        70
   sensitive       0.63      0.60      0.62        63

    accuracy                           0.65       133
   macro avg       0.65      0.64      0.64       133
weighted avg       0.65      0.65      0.65       133

ROC-AUC: 0.7367346938775511
Confusion matrix:
 [[48 22]
 [25 38]]


# **클래스별 예측 확률 이용**
1. predict_proba(클래스별 예측 확률) 이용 전략
2. predict_proba>=0.5인 샘플들 대신 predict_proba>=0.7인 샘플만 선택했을 때의 효과
   - predict_proba가 높을수록 모델이 확신 하는 샘플이며, 임상 분석에서 노이즈를 줄여 임상적 차이가 뚜렷할 것이라 기대

In [15]:
import numpy as np

def threshold_by_class(proba, preds, th_per_class):
    """
    proba        : np.ndarray, shape=(n,2) — predict_proba 결과
    preds        : np.ndarray, shape=(n,)  — predict 결과 (0/1)
    th_per_class : dict — {0: threshold_for_class0, 1: threshold_for_class1}
    
    returns: dict with
      total           : int   — 지정한 클래스별 임계치 중 하나라도 통과한 샘플 수
      counts          : dict  — 통과 샘플의 클래스별 개수
      ratios          : dict  — 통과 샘플의 클래스별 비율
      counts_each_cls : dict  — 각 클래스별 개별 임계치 통과 개수 (AND/OR 구분 없이 단독 통과)
    """
    # 클래스별 mask
    mask0 = proba[:, 0] >= th_per_class[0]
    mask1 = proba[:, 1] >= th_per_class[1]
    # 둘 중 하나라도 통과
    mask_either = mask0 | mask1

    # 통과 샘플수
    total = mask_either.sum()
    
    # 통과 샘플의 예측 레이블
    preds_either = preds[mask_either]
    uniq, cnts = np.unique(preds_either, return_counts=True)
    counts = dict(zip(uniq, cnts))
    ratios = {cls: cnt/total for cls, cnt in counts.items()}
    
    # 각각 클래스별 임계치 단독 통과 개수
    counts_each = {
        0: int(mask0.sum()),
        1: int(mask1.sum())
    }
    
    return {
        "total": total,
        "counts": counts,
        "ratios": ratios,
        "counts_each_cls": counts_each
    }

# ── 사용 예시 ──
proba = voting_clf.predict_proba(X_tcga_final)
preds = voting_clf.predict(X_tcga_final)

res = threshold_by_class(
    proba, preds,
    th_per_class={0:0.6, 1:0.65}
)

print("■ 임계치 중 하나라도 통과한 샘플 수:", res["total"])
print("■ 통과 샘플 클래스별 개수:", res["counts"])
print("■ 통과 샘플 클래스별 비율:", res["ratios"])
print("■ 클래스별 단독 임계치 통과 개수:", res["counts_each_cls"])


■ 임계치 중 하나라도 통과한 샘플 수: 232
■ 통과 샘플 클래스별 개수: {0: 60, 1: 172}
■ 통과 샘플 클래스별 비율: {0: 0.25862068965517243, 1: 0.7413793103448276}
■ 클래스별 단독 임계치 통과 개수: {0: 60, 1: 172}


In [16]:
import numpy as np
import pandas as pd

# 임계치 정의
th0, th1 = 0.6, 0.65

# 1) 예측 확률·레이블 계산
proba = voting_clf.predict_proba(X_tcga_final)   # shape=(n,2)
preds = voting_clf.predict(X_tcga_final)         # shape=(n,)

# 2) 마스크 계산
mask0 = proba[:, 0] >= th0
mask1 = proba[:, 1] >= th1
mask_either = mask0 | mask1

# 3) TCGA_ID와 pred 컬럼만 갖는 DataFrame 생성하되, 마스크로 필터링
pred_simple_df = pd.DataFrame({
    "TCGA_ID": tcga_path.index[mask_either],
    "pred":     preds[mask_either]
})

# 4) CSV 저장 (필터된 샘플만)
pred_simple_df.to_csv('REFAMETINIB_predictions.csv', index=False)
print("Saved", len(pred_simple_df), "rows to tcga_predictions_filtered.csv")


Saved 232 rows to tcga_predictions_filtered.csv


In [17]:
#pred_df.to_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/DASATINIB/TCGA_DASATINIB_predict.csv")

In [19]:
from scipy.stats import fisher_exact
import pandas as pd

# 샘플 타입 추출
pred_simple_df['sample_type'] = pred_simple_df['TCGA_ID'].str[-2:]

# 1. 라벨 매핑
sample_type_map = {'01': '종양', '11': '정상'}
pred_map = {0: '저항', 1: '민감'}

# 2. 01과 11 샘플만 선택
df_01_11 = pred_simple_df[pred_simple_df['sample_type'].isin(['01', '11'])].copy()

# 3. 라벨로 변환
df_01_11['sample_type'] = df_01_11['sample_type'].map(sample_type_map)
df_01_11['pred'] = df_01_11['pred'].map(pred_map)

# 4. 교차표 생성
contingency = pd.crosstab(df_01_11['sample_type'], df_01_11['pred'])

# 5. Fisher’s Exact Test (2×2용 정확 검정)
oddsratio, p = fisher_exact(contingency)

# 6. 출력
print("종양 vs 정상별 민감/저항 분포")
print(contingency)

print("\nFisher’s Exact Test 결과")
print(f"  - p-value: {p:.4f}")
print(f"  - Odds ratio: {oddsratio:.4f}")
if p < 0.05:
    print("  종양과 정상에서 민감/저항 분포가 통계적으로 유의하게 다릅니다.")
else:
    print("  종양과 정상 간 민감/저항 분포 차이는 유의하지 않습니다.")


종양 vs 정상별 민감/저항 분포
pred          민감  저항
sample_type         
정상             8   0
종양           161  59

Fisher’s Exact Test 결과
  - p-value: 0.1163
  - Odds ratio: inf
  종양과 정상 간 민감/저항 분포 차이는 유의하지 않습니다.
